# Домашнее задание 2. Извлечение коллокаций + NER

## Описание

Выберите корпус отзывов на товары одной из категорий Amazon: http://jmcauley.ucsd.edu/data/amazon/

(В низу страницы по ссылке есть код для загрузки данных, можете им воспользоваться)

Допустим, что вам нужно подготовить аналитический отчет по этим отзывам — например, для производителя нового продукта этой категории. Для этого будем искать упоминания товаров в отзывах (будем считать их NE). Учтите, что упоминание может выглядеть не только как "Iphone 10", но и как "модель", "телефон" и т.п.

Важное замечание: в задании приводятся примеры решений, вы можете их использовать!

**1. (3 балла)** Предложите 3 способа найти упоминания товаров в отзывах. Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара. Могут помочь заголовки и дополнительные данные с Amazon (Metadata здесь) Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?

**2. (2 балла)** Реализуйте один из предложенных вами способов.

Примеры в качестве подсказки (можно использовать один из них):

написать правила с помощью natasha/yargy
составить мини-словарь сущностей/дескрипторов, расширить с помощью эмбеддингов (например, word2vec)


**3. (1 балл)** Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

**4. (3 балла)** Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика ранжирует выше коллокации, подходящие для отчёта).

**5. (1 балл)** Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров. Должны получиться примерно такие группы:

watch 


stylish watch, 
good watches, 
great watch, 
love this watch
...

**Бонус (2 балла):** если придумаете способ объединить синонимичные упоминания (например, "Samsung Galaxy Watch", "watch", "smartwatch")

# 1. Предложение способов найти упоминания товаров в отзывах.

## 1.1

Можно выделить самые частотные слова (по абсолютной частоте встречаемости) в названиях товаров и их категории (у нас достаточно много описаний с проставленной категорией). Так мы можем (скорее всего!) выделить названия товаров. Так будет точно меньше "ручной" работы написания "нужных" слов - автоматизация процесса всегда приятнее.

Когда получим "базовые" названия товаров, можно брать 1 или 2 соседа справа, слева, взять би- и три- граммы с "нужным" словом и взять только такие словосочетания, которые встречаются и в названии товаров - так можно отфильтровать "шум", который остался бы после фильтрации просто по частотности, без учета вхождения в название товара

## 1.2

Выделять самые "важные" слова можно и векторайзерами, например,  из просто TFidfVectorizer-а (или как-нибудь еще, это просто к примеру), сделать опять фильтрацию по какому-нибудь порогу частоты встречаемости слова и, может (это будет логичнее и "безопаснее" всего), сделать еще фильрацию по части речи - оставить существительные. Поэтому вряд ли "захватим" таким способом еще и бренды/марки товаров, но что уж тут поделать? Можно поставить "костыль" с поиском самых частотных слов, встречаемых с отобранными существительными.
## 1.3

Никогда не плохо использовать морфологические шаблоны, для этого можно проанализировать, как вообще обычно называют товары (спойлер: обычно NOUN + NOUN, ADJ + NOUN (такой порядок в английском языке), ADJ + NOUN + NOUN), а потом "повытаскивать" из отзывов такие конструкции. В этом подходе есть очевидный недостаток: всегда точно попадутся словосочетания, не соотвествующие цели задания. Но можно поставить "костыль" - например, проверять вхождения слова категории в найденном словосочетании (например, есть ли в выделенной паре слов слово "пылесос"). 

## 1.4

Еще можно "подкрутить" модель, способную распознавать NER-ы, в частности, нас будут интересовать случаи, когда модель сможет находить названия компании - тег ORG, и брать 1 или 2 соседа справа, слева, и, может, по частоте таких би- и три-грамм вычленять NER + NOUN, NOUN + NER самые частые случаи. Тут есть недостаток опять же в том, что не факт, что попадется только то, что нам нужно, но "костыль" с частотностью и дополнительной проверкой на вхождение какого-нибудь слова из списка "нужных" слов (т.е. слов, обозначающие какие-либо бытовые приборы и тд) может частично поправить ситуацию.



**Примечание:** каждый из предложенных способов можно улучшить с помощью  расширения "нужных" словосочетаний с помощью перебора синонимов с word2vec-моделью (например), обученной только на нашем корпусе.

# 2. Реализация одного из предложенных способов.

Для начала импортируем все, что нужно

Здесь же я создаю словарь стоп-слов

In [113]:
import os
import re
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import pymorphy2
import random
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from collections import Counter

import spacy
nlp = spacy.load('en_core_web_sm')
import nltk
from nltk.util import ngrams
from nltk.collocations import *
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import nltk
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')
stopwords_list.extend('img alt src https media amazon com images aplus media vc   d cb  __ jpg _sl300__'.split())
stopwords_list.extend(['img', 'alt', 'src', 'https', 'media', 'amazon',
                       'com', 'images', 'aplus', 'media', 'vc', 'jpg', 'class', 'spacing', 'mini', 'br'])
stopwords_list.extend(['link', 'normal', 'target', '_blank', 'rel', 'noopener', 'href', 'png', 'jpeg',
                      'em', 'bottom', 'line','bfdb'
                      'div', 'text', 'center', 'view', 'larger', 'div'])

In [2]:
# !python -m spacy download en_core_web_sm
# !wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Home_and_Kitchen.json.gz
# !wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Luxury_Beauty.json.gz

https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV


https://colab.research.google.com/drive/12r4KJVbNqjjhiZ6aeiaG809x4-Tg5fm8?usp=sharing#scrollTo=EdaX7BmI8OW_

https://nijianmo.github.io/amazon/index.html

https://nijianmo.github.io/amazon/index.html#files

https://nijianmo.github.io/amazon/index.html#code


https://github.com/named-entity/hse-nlp/blob/master/4th_year/hw/hw2.md

In [3]:
meta_data = []

i = 0
with gzip.open('meta_Home_and_Kitchen.json.gz', 'r') as f:
# with open('meta_Home_and_Kitchen.json') as f:
    for l in tqdm(f):
        if i < 10000:
            meta_data.append(json.loads(l))
            i += 1
        else:
            break

9666it [00:02, 4082.41it/s]


In [4]:
data = []

i = 0
with open('sample_Home_and_Kitchen_5.json') as f:
    for l in tqdm(f):
        if i < 10000:
            data.append(json.loads(l))
        else:
            break

41047it [00:00, 82371.93it/s]


In [5]:
data[0]

{'overall': 4.0,
 'verified': True,
 'reviewTime': '08 3, 2014',
 'reviewerID': 'A2LVMEG7IP5F90',
 'asin': 'B00002N62Y',
 'reviewerName': 'jeff',
 'reviewText': 'good product and price',
 'summary': 'Four Stars',
 'unixReviewTime': 1407024000}

In [6]:
reviewText = []
asin = []
progress_bar = tqdm(total=len(data))
for i,d in enumerate(data):
    if 'reviewText' in d.keys():
        reviewText.append(d['reviewText'])
        asin.append(d['asin'])
    progress_bar.update(1)
progress_bar.close()

100%|██████████| 41047/41047 [00:00<00:00, 504257.50it/s]


In [21]:
df = pd.DataFrame({'reviewText': reviewText, 'asin': asin}).head(10000)
df.head(3)

,reviewText,asin
0,good product and price,B00002N62Y
1,These belts are $2+ retail and some retailers/...,B00002N62Y
2,These came in a 2 pk and are perfect fit for m...,B00002N62Y


In [8]:
description = []
title = []
category = []
brand = []
category_correct = []
categories_to_count = []
asin2 = []

def all_categories(list_cat):
    ans = []
    for cat in list_cat:
        if '&' in cat:
            ans.extend(cat.split(' & '))
        else:
            ans.append(cat)
    return ans


progress_bar = tqdm(total=len(meta_data))
for i,d in enumerate(meta_data):
    # описания товаров лежат списком из предложений - я сохраню все строчкой
    if 'description' in d.keys():
        description.append(' '.join(d['description']))
        title.append(d['title'])
        if d['category']:
        # берем все элементы списка, кроме первого - тк там 
        # лежит "главная" категория - Home nad Kitchen
            category.append(d['category'][1:])
            cat = all_categories(d['category'][1:])
            category_correct.append(set(cat))
            categories_to_count.extend(list(set(cat)))
        else:
            category.append('no category')
        if 'brand' in d.keys():
            brand.append(d['brand'])
        else:
            brand.append('no brand')

        asin2.append(d['asin'])
    progress_bar.update(1)
progress_bar.close()



meta = pd.DataFrame({'description': description,
                   'category': category,
                   'category_correct': category_correct,
                  'title': title,
                  'brand': brand,
                'asin': asin2})

100%|██████████| 10000/10000 [00:00<00:00, 35358.81it/s]


In [9]:
meta.head(5)

,description,category,category_correct,title,brand,asin
0,It was a time honored tradition among the earl...,"[Kitchen & Dining, Dining & Entertaining, Dinn...","{Kitchen, Plates, Dinner Plates, Dining, Enter...",You Are Special Today Red Plate [With Red Pen],Waechtersbach USA,0001487795
1,VICKS INHALER relieves stuffy noses helps sinu...,"[Home Dcor, Candles & Holders, Candles]","{Holders, Candles, Home Dcor}",Vicks Inhaler Relief for Cold Sinus Nasal Cong...,Vicks,0002020300
2,"16 oz squeeze bottle, 1 lb.","[Kitchen & Dining, Dining & Entertaining, Glas...","{Drinkware, Kitchen, Wine, Champagne Glasses, ...",Artistic Churchware Communion Cup Filler: RW525,Artistic Churchware,0006564224
3,The only soap in the world with a unique combi...,"[Bath, Bathroom Accessories]","{Bathroom Accessories, Bath}",4 BARS! Mysore Sandal Soap 70grams FAST SHIPPING,Mysore,0009046461
4,"Divya Arogya vati improves health, It has natu...","[Home Dcor, Home Fragrance, Incense & Incense ...","{Home Fragrance, Incense Holders, Home Dcor, I...",AROGYA VATI (40gm) by popeye seller,Patanjali,0234937912


In [10]:
categories_Counter = Counter(categories_to_count)

In [11]:
np.mean(list(categories_Counter.values()))

48.7314990512334

In [12]:
threshold = 10
categories_Counter_filtered = {key: value for key,value in categories_Counter.items() if value > threshold}

In [13]:
meta.shape

(10000, 6)

In [14]:
print(meta['description'][1000], '\n', meta['title'][1000])

Perfect for cheesecake and other cake baking, this Kaiser Bakeware Noblesse Nonstick Springform Pan features a heavy gauge steel for even heat distribution and consistent browning, while the non-stick interior eliminates the need for flouring pans. The buckle allows you to remove the exterior of the pan without compromising the cake shape, rendering a perfect circumference every time. This leakproof 10-inch springform pan is ideal for that treasured family cheesecake recipe. The pan's walls are reversible for both right- and left-handed use, and its superior buckle has a lifetime warranty. The perfectly flat bottom is easy to release and provides an evenly baked dessert. For optimal cooking, coat the interior surface of the pan with a very thin layer of butter before each use--this will guarantee a perfect release of cakes and breads. Because of this pan's heavy-gauge steel construction and dark-colored coating, goods baked in it will generally finish cooking in about 10% less time tha

In [15]:
print(df['reviewText'][1000])

Our Kenmore canister vacuum broke down after only a few years, and we were pretty frustrated. We figured that if every vacuum we buy is going to break down after a year or two, we might as well buy something more affordable! So far, I think this vacuum is working well for us. (I'll update if it breaks down).

So things to know: it does look cheap. The plastic parts are clunky. But, it is light-weight and easy to carry around the house, and it seems to be picking up everything efficiently. I was able to maneuver it under furniture more easily than the canister. Unlike other reviewers, I don't find it to be particularly noisy (at least, no more so then the Kenmore that cost 4x as much!).  It also did a good job on cat hair in high usage areas.


In [16]:
def bt(token_pattern=r'(?u)\b\w\w+\b'):
    return re.compile(token_pattern).findall
tokenizer=bt()
print(tokenizer('hello how are you ?.mwlkdj----- +78231'))
 
def tokenize_line(line: str):
    if isinstance(line, str):
        return [w.lower() for w in tokenizer(line)]
    else:
        return 'нет текста'
print(tokenize_line('hello how are you ?.mwlkdj----- +78231'))
 

# class Lemmatizer:
#     def __init__(self):
#         self.morph = pymorphy2.MorphAnalyzer()
      
#     def __call__(self, x: str) -> str:
#         lemma = self.morph.parse(x)[0].normal_form
#         return lemma
    
lemmatizer = WordNetLemmatizer()
 
def lemmatize_df(df, text_column):
    unique_toks = set([])
    for line in tqdm(df[text_column]):
        for w in line:
            if w not in unique_toks:
                unique_toks.add(w)
    print('made unique tokens set')
    t_l = {}
    for ut in unique_toks:
        t_l[ut] = lemmatizer.lemmatize(ut)
    print('made tokens-lemmas dict')
    return unique_toks, t_l
 
def lemmatize_line(t_l, line):
    return [t_l[w] for w in line if w in t_l]

def clean_lemmas(line):
    ans = []
    for w in line:
        if re.findall('_\w\w[0-9]*__', w) == []:
            if re.findall('\d', w) == []:
                if w not in stopwords_list:
                    ans.append(w)
    return ans

['hello', 'how', 'are', 'you', 'mwlkdj', '78231']
['hello', 'how', 'are', 'you', 'mwlkdj', '78231']


In [17]:
lemmatizer.lemmatize('bees')

'bee'

In [22]:
df['tokens'] = df['reviewText'].apply(tokenize_line)
unique_toks_df, t_l_df = lemmatize_df(df, 'tokens')
df['lemmas'] = df['tokens'].apply(lambda x: lemmatize_line(t_l_df, x))
df['cleaned_lemmas'] = df['lemmas'].apply(clean_lemmas)
df['cleaned_lemmas_str'] = df['cleaned_lemmas'].apply(lambda x: ' '.join(x))

100%|██████████| 10000/10000 [00:00<00:00, 103916.38it/s]


made unique tokens set
made tokens-lemmas dict


In [23]:
df.head()

,reviewText,asin,tokens,lemmas,cleaned_lemmas,cleaned_lemmas_str
0,good product and price,B00002N62Y,"[good, product, and, price]","[good, product, and, price]","[good, product, price]",good product price
1,These belts are $2+ retail and some retailers/...,B00002N62Y,"[these, belts, are, retail, and, some, retaile...","[these, belt, are, retail, and, some, retailer...","[belt, retail, retailer, seller, use, generic,...",belt retail retailer seller use generic non oe...
2,These came in a 2 pk and are perfect fit for m...,B00002N62Y,"[these, came, in, pk, and, are, perfect, fit, ...","[these, came, in, pk, and, are, perfect, fit, ...","[came, pk, perfect, fit, good, old, vac, seem,...",came pk perfect fit good old vac seem strong d...
3,"So my super fancy vacuum cleaner, the one with...",B00002N62Y,"[so, my, super, fancy, vacuum, cleaner, the, o...","[so, my, super, fancy, vacuum, cleaner, the, o...","[super, fancy, vacuum, cleaner, one, cold, fus...",super fancy vacuum cleaner one cold fusion rea...
4,Right one for my eureka.,B00002N62Y,"[right, one, for, my, eureka]","[right, one, for, my, eureka]","[right, one, eureka]",right one eureka


In [24]:
df.shape

(10000, 6)

In [25]:
meta = meta[meta['description'] != 'no description'].reset_index()
meta['tokens'] = ['no text'] * meta.shape[0]
for i, com in enumerate(meta['description']):
    if isinstance(com, str):
        meta['tokens'][i] = tokenize_line(com)
unique_toks, t_l = lemmatize_df(meta, 'tokens')
meta['lemmas'] = meta['tokens'].apply(lambda x: lemmatize_line(t_l, x))
meta['cleaned_lemmas'] = meta['lemmas'].apply(clean_lemmas)
df_orig = meta.copy()
meta['cleaned_lemmas_str'] = meta['cleaned_lemmas'].apply(lambda x: ' '.join(x))
meta = meta.drop_duplicates('cleaned_lemmas_str')
meta.head(1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 10000/10000 [00:00<00:00, 54492.92it/s]


made unique tokens set
made tokens-lemmas dict


,index,description,category,category_correct,title,brand,asin,tokens,lemmas,cleaned_lemmas,cleaned_lemmas_str
0,0,It was a time honored tradition among the earl...,"[Kitchen & Dining, Dining & Entertaining, Dinn...","{Kitchen, Plates, Dinner Plates, Dining, Enter...",You Are Special Today Red Plate [With Red Pen],Waechtersbach USA,0001487795,"[it, was, time, honored, tradition, among, the...","[it, wa, time, honored, tradition, among, the,...","[wa, time, honored, tradition, among, early, a...",wa time honored tradition among early american...


In [26]:
for i,t in enumerate(meta['cleaned_lemmas_str']):
    if  i < 2:
        print(f'{i}) ', meta['title'][i], '\n', ', '.join(meta['category'][i]), '\n', t, '\n\n')
    else:
        break

0)  You Are Special Today Red Plate [With Red Pen] 
 Kitchen & Dining, Dining & Entertaining, Dinnerware, Plates, Dinner Plates 
 wa time honored tradition among early american family someone deserved special praise attention served dinner red plate observe special occasion 


1)  Vicks Inhaler Relief for Cold Sinus Nasal Congestion Allergy 
 Home Dcor, Candles & Holders, Candles 
 vicks inhaler relief stuffy nose help sinus congestion breathe easy great allergy season made india 




Пособираем самые частотные существительные корпуса:

In [80]:
def nouns_counter(lines_list):
    counter = []
    progress_bar = tqdm(total=len(lines_list))
    for line in lines_list:
        line = nlp(line)
        to_add = [t.text for t in line if t.text not in stopwords_list and t.pos_ == 'NOUN']
        counter.extend(to_add)
        progress_bar.update(1)
    progress_bar.close()
    return Counter(counter)

In [47]:
c = nouns_counter(df.cleaned_lemmas_str)

In [50]:
c.most_common(10)

[('vacuum', 12543),
 ('dyson', 3113),
 ('year', 3003),
 ('suction', 2911),
 ('floor', 2903),
 ('carpet', 2831),
 ('time', 2759),
 ('hose', 2618),
 ('bag', 2325),
 ('use', 2134)]

В целом, есть тут то, что нам нужно, это хорошо, но слова 'use', 'year', 'suction и 'time' кажутся тут лишними. Значит, попробуем сделать то же самое со словами из метадаты.

In [56]:
# c2 = nouns_counter(meta.cleaned_lemmas_str)

In [57]:
c2.most_common(10)

[('inch', 8096),
 ('steel', 6228),
 ('knife', 4632),
 ('pan', 4224),
 ('piece', 3698),
 ('food', 3293),
 ('handle', 2976),
 ('hand', 2901),
 ('heat', 2882),
 ('feature', 2811)]

Тут тоже встречается много "мусора"

In [75]:
def cat_preproc(list_line):
    ans = []
    for line in list_line:
        line = re.sub(' &', '', line)
        line = lemmatizer.lemmatize(line)
        add = [w.lower() for w in line.split() if w not in stopwords_list]
        ans.extend(add)
    return ans

cat_count = []
for c in meta.category:
     cat_count.extend(c)
cat_counter = Counter(cat_count)
cat_counter.most_common(10)

In [64]:
cat_counter_nouns = nouns_counter(list(cat_counter.keys()))

In [65]:
cat_counter_nouns.most_common(10)

[('Sets', 33),
 ('Knives', 26),
 ('Accessories', 12),
 ('Glasses', 11),
 ('Dishes', 8),
 ('Bags', 7),
 ('Bowls', 6),
 ('Covers', 6),
 ('Ovens', 6),
 ('Candy', 5)]

Урааа, вот это то, что нужно!!! Надо было только лемматизировать, сейчас поправим)

In [81]:
cat_count_prec = cat_preproc(cat_count)
cat_counter_prec = Counter(cat_count_prec)
cat_counter_prec.most_common(10)

[('kitchen', 8474),
 ('dining', 8338),
 ('utensils', 1739),
 ('accessories', 1555),
 ('gadgets', 1501),
 ('entertaining', 1422),
 ('cookware', 1235),
 ('tools', 1055),
 ('sets', 1037),
 ('knife', 962)]

Так точно лучше, и счет немного другой, оставляю даже без шлифовки существительных

Еще с TFidf посчитаем частотные биграммы, и будет красота!

In [97]:
cv = CountVectorizer(max_df=0.75, stop_words=stopwords_list, max_features=1000, ngram_range=(2,2))
word_count_vector = cv.fit_transform(df.cleaned_lemmas_str.to_list())

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [98]:
def topn(feats, sorted_items, threshold=0.1):    
    sorted_items = [item for item in sorted_items if item[1] > threshold]  
    score_vals = []
    feature_vals = []
    for idx, score in sorted_items:
        fname = feats[idx]
        score_vals.append(round(score, 3))
        feature_vals.append(feats[idx])
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    return results

def sort_coo_matrix(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def important_features(text, threshold=0.1):
    feats = cv.get_feature_names()
    tf_idf_vector = tfidf_transformer.transform(cv.transform([text]))
    sorted_items = sort_coo_matrix(tf_idf_vector.tocoo())
    keywords = topn(feats, sorted_items, threshold)
    return set(keywords.keys())

In [102]:
imp_feats = important_features(' '.join(meta.title.to_list()), threshold=0.05)
imp_feats

{'area rug',
 'bagless upright',
 'black decker',
 'canister vacuum',
 'carpet cleaner',
 'dirt devil',
 'easy clean',
 'five star',
 'hand held',
 'hand vac',
 'handheld vacuum',
 'heavy duty',
 'hepa filter',
 'type vacuum',
 'upright vacuum',
 'vacuum cleaner',
 'whole house'}

Огонь!

In [110]:
all_titles = [key for key,value in cat_counter_prec.items() if value > np.mean(list(cat_counter_prec.values()))]
all_titles.extend(imp_feats)
all_titles[:10]

['kitchen',
 'dining',
 'entertaining',
 'dinnerware',
 'plates',
 'home',
 'dcor',
 'candles',
 'holders',
 'glassware']

In [126]:
bigrams = ngrams(' '.join(df.cleaned_lemmas_str).split(), 2)

In [127]:
bigrams_counter = Counter(bigrams)
bigrams_counter.most_common(10)

[(('vacuum', 'cleaner'), 1346),
 (('work', 'great'), 706),
 (('work', 'well'), 611),
 (('pet', 'hair'), 501),
 (('dirt', 'devil'), 428),
 (('easy', 'use'), 407),
 (('light', 'weight'), 373),
 (('bare', 'floor'), 362),
 (('hardwood', 'floor'), 342),
 (('dog', 'hair'), 321)]

In [128]:
bigrams_filtered = []
for b in bigrams_counter.keys():
    if b[0] in all_titles or b[1] in all_titles:
        bigrams_filtered.append(b)

In [129]:
bigrams_filtered[0]

('fancy', 'vacuum')

In [163]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
# # Экстрактор коллокаций
finder2 = BigramCollocationFinder.from_documents(df.cleaned_lemmas)
finder2.apply_freq_filter(5)
#finder2.score_ngrams(bigram_measures.likelihood_ratio)

In [166]:
pmi = []
for t in all_titles:
    rate = [i for i in finder2.nbest(bigram_measures.pmi, 10000) if t in i]
    pmi.append([' '.join(i) for i in rate])
pmi[0][:8]

['permitted kitchen',
 'pasta kitchen',
 'random kitchen',
 'kitchen counter',
 'kitchen trashcan',
 'tiled kitchen',
 'minus kitchen',
 'kitchen cabinet']

In [168]:
jaccard = []
for t in all_titles:
    rate = [i for i in finder2.nbest(bigram_measures.jaccard, 10000) if t in i]
    jaccard.append([' '.join(i) for i in rate])
jaccard[0][:8]

['kitchen cabinet',
 'kitchen bathroom',
 'tiled kitchen',
 'kitchen counter',
 'kitchen trashcan',
 'bathroom kitchen',
 'please kitchen',
 'kitchen tiled']

In [170]:
dice = []
for t in all_titles:
    rate = [i for i in finder2.nbest(bigram_measures.dice, 10000) if t in i]
    dice.append([' '.join(i) for i in rate])
dice[0][:8]

['kitchen cabinet',
 'kitchen bathroom',
 'tiled kitchen',
 'kitchen counter',
 'kitchen trashcan',
 'bathroom kitchen',
 'please kitchen',
 'kitchen tiled']

Более-менее похожие выдачи с метрик, оставим  pmi, мне она понравилась больше всего.

In [172]:
likelihood_ratio = []
for t in all_titles:
    rate = [i for i in finder2.nbest(bigram_measures.likelihood_ratio, 10000) if t in i]
    likelihood_ratio.append([' '.join(i) for i in rate])
likelihood_ratio[0][:8]

['kitchen cabinet',
 'kitchen bathroom',
 'kitchen floor',
 'tiled kitchen',
 'kitchen counter',
 'kitchen living',
 'room kitchen',
 'kitchen trashcan']

In [173]:
ans = {}
for idx, item in enumerate(all_titles[:5]):
    ans[item] = jaccard[idx][:5] + likelihood_ratio[idx][:5]

In [188]:
ans['kitchen']

['kitchen cabinet', 'kitchen bathroom', 'tiled kitchen', 'kitchen counter', 'kitchen trashcan', 'kitchen cabinet', 'kitchen bathroom', 'kitchen floor', 'tiled kitchen', 'kitchen counter']


In [187]:
ans['dining']

['den dining', 'dining room', 'dining table', 'bathroom dining', 'dining chair', 'dining room', 'room dining', 'den dining', 'living dining']


In [190]:
ans['plate']

['blue plate', 'plate kitchen', 'dining plate', 'pocelain plate', 'wedding plate']


In [191]:
print(['metal knife', 'bread knife', 'fish knife', 'steel knife', 'knife kitchen'])

['metal knife', 'bread knife', 'fish knife', 'steel knife', 'knife kitchen']


Не могу сказать, что мне очень нравится то, что получилось. Есть, так сказать, к чему стремится.

Я бы это связала и с самими отзывами: люди чаще описывают не сам товар, а как бы то, что связано с выбором этого товара: цена, качество, что тоже верно. Время использования или покупки, это полезная информация для дальнейшей автоматической оценки товара, даже для понимания периода его использования - christmas всякие штуки - зимой, мб какая-нибудь подставка под индейку - в июле и тд. Это и так очевидно жиому человеку, но не всегда машине. Или понимать, что скатерти нормально прослужить пару лет, а вот пылесос пусть бы работал более 10 лет.


Как можно улучшить качество?

Я бы точно посмотрела триграммы, как и хотела (но так расстроилась в биграммах, что забыла про это)

Точно бы больше поработала с изначальной фильтрацией слов, которые мы считаем важными: может быть, что-то и ручками пописать и подополнять корпус важных слов с помощью word2vec, вообще поискала бы всякие NER-штуки в тексте и тоже бы их подектектила бы - это точно помогло бы нам.